In [3]:
import pandas as pd

# Data exploration for PRC Challenge 2025


## Files


In [8]:
pd.read_parquet("../data/apt.parquet").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8787 entries, 0 to 8786
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   icao       8787 non-null   object 
 1   longitude  8787 non-null   float64
 2   latitude   8787 non-null   float64
 3   elevation  8568 non-null   float64
dtypes: float64(3), object(1)
memory usage: 274.7+ KB


In [10]:
pd.read_parquet("../data/flightlist_train.parquet").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11037 entries, 0 to 11036
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   flight_date       11037 non-null  object        
 1   aircraft_type     11037 non-null  object        
 2   takeoff           11037 non-null  datetime64[us]
 3   landed            11037 non-null  datetime64[us]
 4   origin_icao       11037 non-null  object        
 5   origin_name       11037 non-null  object        
 6   destination_icao  11037 non-null  object        
 7   destination_name  11037 non-null  object        
 8   flight_id         11037 non-null  object        
dtypes: datetime64[us](2), object(7)
memory usage: 776.2+ KB


In [7]:
pd.read_parquet("../data/flights_train/prc770822360.parquet").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25461 entries, 0 to 25460
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   timestamp      25461 non-null  datetime64[ns]
 1   flight_id      25461 non-null  object        
 2   typecode       25461 non-null  object        
 3   latitude       25461 non-null  float64       
 4   longitude      25461 non-null  float64       
 5   altitude       25453 non-null  float64       
 6   groundspeed    18078 non-null  float64       
 7   track          18078 non-null  float64       
 8   vertical_rate  18078 non-null  float64       
 9   mach           8 non-null      float64       
 10  TAS            0 non-null      float64       
 11  CAS            0 non-null      float64       
 12  source         25461 non-null  object        
dtypes: datetime64[ns](1), float64(9), object(3)
memory usage: 2.5+ MB


In [9]:
pd.read_parquet("../data/fuel_train.parquet").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131530 entries, 0 to 131529
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   idx        131530 non-null  int64         
 1   flight_id  131530 non-null  object        
 2   start      131530 non-null  datetime64[ns]
 3   end        131530 non-null  datetime64[ns]
 4   fuel_kg    131530 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(1), object(1)
memory usage: 5.0+ MB


## Flight list


In [ ]:
def empty_str(s):
    return (s.str.len() == 0) | s.isna()


flightlist_df = pd.read_parquet("../data/flightlist_train.parquet").assign(
    duration=lambda df: df["landed"] - df["takeoff"],
)

for col in ("origin_icao", "destination_icao", "aircraft_type"):
    print(f"Missing {col}: {empty_str(flightlist_df[col]).sum()}")

print(f"\nDuration --- {flightlist_df['duration'].isna().sum()} missing")
print(flightlist_df["duration"].describe())

Missing origin_icao: 0
Missing destination_icao: 0
Missing aircraft_type: 0

Duration --- 0 missing
count                     11037
mean     0 days 05:32:29.436894
std      0 days 03:28:10.673887
min             0 days 00:20:14
25%             0 days 02:57:45
50%             0 days 04:06:23
75%             0 days 08:28:06
max             0 days 17:31:28
Name: duration, dtype: object


## APT (airports)


In [ ]:
apt_df = pd.read_parquet("../data/apt.parquet")
for col in ("longitude", "latitude", "elevation"):
    print(
        f"Range of {col}: [{apt_df[col].min()}, {apt_df[col].max()}] - {apt_df[col].isna().sum()} missing"
    )

apt_all_icao = set(apt_df["icao"].unique())
flightlist_all_icao = set(flightlist_df["destination_icao"].unique()) | set(
    flightlist_df["destination_icao"].unique()
)
missing_icao = flightlist_all_icao - apt_all_icao
if missing_icao:
    print(f"The following ICAO in flight list are missing from APT: {missing_icao}")
else:
    print("All ICAO in flight list are in APT")

Range of longitude: [-179.876999, 179.9757] - 0 missing
Range of latitude: [-79.777778, 82.517799] - 0 missing
Range of elevation: [-1266.0, 14472.0] - 219 missing
All ICAO in flight list are in APT


## Fuel


In [142]:
fuel_df = pd.read_parquet("../data/fuel_train.parquet").assign(
    duration=lambda df: df["end"] - df["start"],
    fuel_consumption_kg_h=lambda df: df["fuel_kg"]
    / (df["duration"].dt.total_seconds() / 3600),
)
fuel_fl_merged = pd.merge(
    fuel_df, flightlist_df, how="left", on="flight_id", suffixes=("", "_fl")
)

for col in ("duration", "fuel_consumption_kg_h"):
    print(f"{col} --- {fuel_df[col].isna().sum()} missing")
    print(fuel_df[col].describe())
    print()

missing_from_flight_list = set(
    fuel_df[fuel_fl_merged["start"].isna()]["flight_id"].unique()
)
if missing_from_flight_list:
    print(
        f"The following segment flight IDs are missing from flight_list: {missing_from_flight_list}"
    )
else:
    print("All segment flight IDs are in flight_list")
print()

td0 = pd.Timedelta(0)
start_after_land = fuel_fl_merged["start"] - fuel_fl_merged["landed"]
start_after_land_consumption = fuel_df["fuel_consumption_kg_h"][start_after_land > td0]
end_before_takeoff = fuel_fl_merged["takeoff"] - fuel_fl_merged["end"]
end_before_takeoff_consumption = fuel_df["fuel_consumption_kg_h"][
    end_before_takeoff > td0
]
print(
    f"Segments starting after landed: {(start_after_land > td0).sum()} (max {start_after_land.max()})"
)
print(
    f"Consumption: [{start_after_land_consumption.min()}, {start_after_land_consumption.max()}] avg {start_after_land_consumption.mean()}"
)
print()
print(
    f"Segments ending before takeoff: {(end_before_takeoff > td0).sum()} (max {end_before_takeoff.max()})"
)
print(
    f"Consumption: [{end_before_takeoff_consumption.min()}, {end_before_takeoff_consumption.max()}] avg {end_before_takeoff_consumption.mean()}"
)

# TODO compute time outside intervals per flight

duration --- 0 missing
count                       131530
mean     0 days 00:08:51.938028556
std      0 days 00:09:19.682244268
min         0 days 00:00:01.610000
25%         0 days 00:04:56.603000
50%         0 days 00:05:00.582000
75%         0 days 00:10:00.852000
max         0 days 00:59:59.993000
Name: duration, dtype: object

fuel_consumption_kg_h --- 0 missing
count    131530.000000
mean       3354.816981
std        2739.599780
min          35.620499
25%        1198.725355
50%        2400.728221
75%        5160.641066
max       59368.321064
Name: fuel_consumption_kg_h, dtype: float64

All segment flight IDs are in flight_list

Segments starting after landed: 855 (max 0 days 02:44:11.572000)
Consumption: [51.7372536594639, 25842.83476031462] avg 2817.993561624018

Segments ending before takeoff: 1824 (max 0 days 03:54:55.050000)
Consumption: [114.55572883453402, 49532.604837134044] avg 3745.4379688352487


## Trajectories

Each flight has a different trajectory file (`data/flights_train/{flight_id}.parquet`)


In [ ]:
from pathlib import Path

traj_files = {p.stem for p in Path("../data/flights_train/").glob("*.parquet")}
flightlist_all_flightids = set(flightlist_df["flight_id"].unique())
missing_traj_files = flightlist_all_flightids - traj_files
if missing_traj_files:
    print(
        f"The following flight IDs don't have a corresponding trajectory file: {missing_traj_files}"
    )
else:
    print("All flights have a trajectory file")

All flights have a trajectory file


In [124]:
def stats_trajfile(flight_id: str, ts_tol_min=5):
    stats = {}
    traj_df = pd.read_parquet(f"../data/flights_train/{flight_id}.parquet").assign(
        ts_diff=lambda df: df["timestamp"].diff(),
        latitude_rate_dd_s=lambda df: df["latitude"].diff()
        / df["ts_diff"].dt.total_seconds(),
        longitude_rate_dd_s=lambda df: df["longitude"].diff()
        / df["ts_diff"].dt.total_seconds(),
        altitude_rate_dd_s=lambda df: df["altitude"].diff()
        / df["ts_diff"].dt.total_seconds(),
    )

    assert not traj_df["timestamp"].isna().any()
    stats["traj_n"] = traj_df["timestamp"].count()
    stats["traj_start"] = traj_df["timestamp"].min()
    stats["traj_end"] = traj_df["timestamp"].max()

    missing_ts = traj_df["ts_diff"] > pd.offsets.Minute(ts_tol_min)
    stats["traj_missing_n"] = missing_ts.sum()
    stats["traj_missing_dur"] = traj_df["ts_diff"][missing_ts].sum()

    return stats


stats_trajfile("prc770822360")

{'traj_n': np.int64(25461),
 'traj_start': Timestamp('2025-04-12 16:16:35.464999914'),
 'traj_end': Timestamp('2025-04-13 04:33:32.569999933'),
 'traj_missing_n': np.int64(3),
 'traj_missing_dur': Timedelta('0 days 08:09:39.094000100')}

In [129]:
traj_stats = pd.DataFrame.from_dict(
    list(flightlist_df["flight_id"].apply(stats_trajfile))
)

In [140]:
traj_stats.assign(
    missing_start=(traj_stats["traj_start"] - flightlist_df["takeoff"]).clip(td0),
    missing_end=(flightlist_df["landed"] - traj_stats["traj_end"]).clip(td0),
).describe()

,traj_n,traj_start,traj_end,traj_missing_n,traj_missing_dur,missing_start,missing_end
count,11037.000000,11037,11037,11037.000000,11037,11037,11037
mean,11197.571442,2025-06-02 19:35:32.321972736,2025-06-03 00:12:03.194041856,4.167346,0 days 02:23:00.663822777,0 days 00:30:40.040173417,0 days 00:25:47.819215456
min,17.000000,2025-04-12 16:16:35.464999914,2025-04-12 22:22:30.019000053,0.000000,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00
25%,6553.000000,2025-05-02 01:25:12.936999936,2025-05-02 04:27:09.828999936,0.000000,0 days 00:00:00,0 days 00:00:15.783000,0 days 00:00:25.808000088
50%,9871.000000,2025-05-26 12:12:52.950000128,2025-05-26 18:49:10.479000064,2.000000,0 days 00:51:44.114000023,0 days 00:00:48.686000109,0 days 00:00:57.753999949
75%,15202.000000,2025-07-03 01:43:05.154000128,2025-07-03 04:14:11.658999808,7.000000,0 days 03:55:58.457000172,0 days 00:29:01.107000113,0 days 00:12:37.911999941
max,42114.000000,2025-09-01 02:14:43.378999949,2025-09-01 05:57:35.762000084,34.000000,0 days 15:17:03.728999853,0 days 13:04:50.289000034,0 days 12:22:36.092999935
std,6676.801756,NaN,NaN,5.290252,0 days 03:06:52.717593677,0 days 01:03:13.629407266,0 days 00:55:47.606020401
